## Race Condition

In [1]:
import time
import threading
import concurrent.futures

In [2]:
class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        print(f"{name}: starting update")
        local_copy = self.value
        local_copy += 10
        time.sleep(0.1)
        self.value = local_copy
        print(f"{name}: finishing update")

In [3]:
db = FakeDatabase()
print(f"Testing update. Starting value is {db.value}.")
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(db.update, ("thread 1", "thread 2"))

print(f"Testing update. Ending value is {db.value}.")

Testing update. Starting value is 0.
thread 1: starting update
thread 2: starting update
thread 1: finishing update
thread 2: finishing update
Testing update. Ending value is 10.


In [4]:
# Output must be 20 , but it is 10 !!!!
# It's because race condition

---

## Solving Rase Condition by `Lock`

In [5]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        self._lock.acquire()  ## get the lock
        print(f"{name}: starting update")
        local_copy = self.value
        local_copy += 10
        time.sleep(0.1)
        self.value = local_copy
        print(f"{name}: finishing update")
        self._lock.release()  ## release the lock

In [6]:
db = FakeDatabase()
print(f"Testing update. Starting value is {db.value}.")
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(db.update, ("thread 1", "thread 2"))
    
print(f"Testing update. Ending value is {db.value}.")

Testing update. Starting value is 0.
thread 1: starting update
thread 1: finishing update
thread 2: starting update
thread 2: finishing update
Testing update. Ending value is 20.


## Best Practice is using `Lock` with `with`

In [7]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        with self._lock:
            print(f"{name}: starting update")
            local_copy = self.value
            local_copy += 10
            time.sleep(1)
            self.value = local_copy
            print(f"{name}: finishing update")

In [8]:
db = FakeDatabase()
print(f"Testing update. Starting value is {db.value}.")
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(db.update, ("thread 1", "thread 2"))
print(f"Testing update. Ending value is {db.value}.")

Testing update. Starting value is 0.
thread 1: starting update
thread 1: finishing update
thread 2: starting update
thread 2: finishing update
Testing update. Ending value is 20.


## RED NOTE!!!
If your function has an error, it does not raise if you don't use the output of `executer` because it is a generator. To see this issue, you can run the above script without importing `time`.